# Wroclaw Public Transport

***

#### About the notebook
In this notebook, I will make some geographical analysis of the public transport in Wroclaw, Poland.

<br>

***

#### About the dataset
This GPS dataset is from [www.kaggle.com](https://www.kaggle.com). It was collected by the user [Piotr](https://www.kaggle.com/pieca111). It represents data from public transport vehicles in Wrocław, Poland in the period 2022-04-13 to 2022-04-30.

***

### (1) Import data from CSV

In [69]:
# IMPORTS
import pandas as pd

# LOAD CSV IN DATAFRAME
df = pd.read_csv("../data/positions.csv",
                 dtype='unicode',
                 names=["ID",
                        "Vehicle-ID",
                        "Line",
                        "Type",
                        "Latitude",
                        "Longitude",
                        "Timestamp",
                        ])

# DELETE FIRST ROW
df = df.iloc[1: , :]

### (2) Print dataframe to explore data

In [70]:
df.head(100)

,ID,Vehicle-ID,Line,Type,Latitude,Longitude,Timestamp
1,1,19707402,33,tram,51.113544,17.067019,2022-04-13T11:34:46.810755
2,2,19707424,33,tram,51.11554,17.074562,2022-04-13T11:34:46.810755
3,3,19679615,31,tram,51.112755,17.012657,2022-04-13T11:34:46.810755
4,4,19679761,31,tram,51.112915,17.01011,2022-04-13T11:34:46.810755
5,5,19679724,31,tram,51.12958,16.982096,2022-04-13T11:34:46.810755
...,...,...,...,...,...,...,...
96,96,19767216,d,bus,51.15687,17.121798,2022-04-13T11:34:46.810755
97,97,19767441,k,bus,7158.279,7158.279,2022-04-13T11:34:46.810755
98,98,19708471,70,tram,51.121685,17.043598,2022-04-13T11:34:46.810755
99,99,19764541,131,bus,51.14542,17.108097,2022-04-13T11:34:46.810755


#### Split and convert 'Timestamp' into 'Date' and 'Time'

In [71]:
# SPLIT 'TIMESTAMP' COLUMN
df[['Date', 'Time']] = df['Timestamp'].str.split('T', 1, expand=True)

In [72]:
# CONVERT 'DATE' COLUMN
df['Date'] = pd.to_datetime(df.Date, format='%Y-%m-%d')

In [73]:
# CONVERT 'TIME' COLUMN
df['Time'] = pd.to_datetime(df.Time, format ='%H:%M:%S.%f', errors = 'coerce').dt.time # TIME IS STILL AN OBJECT

In [74]:
# REMOVE CHARACTER IN 'TIMESTAMP' COLUMN
df['Timestamp'] = df["Timestamp"].str.replace("T"," ")

In [75]:
# CONVERT 'TIMESTAMP' COLUMN
df['Timestamp'] = pd.to_datetime(df.Timestamp, format='%Y-%m-%d %H:%M:%S.%f')

#### Convert Geo-Positions (Latitude, Longitude)


In [76]:
# CONVERT LATITUDE
df = df.astype({'Latitude':'float'})
# CONVERT LONGITUDE
df = df.astype({'Longitude':'float'})

In [77]:
# PRINT DTYPES OF DATAFRAME
df.dtypes

ID                    object
Vehicle-ID            object
Line                  object
Type                  object
Latitude             float64
Longitude            float64
Timestamp     datetime64[ns]
Date          datetime64[ns]
Time                  object
dtype: object

#### Remove Outliers (Latitude, Longitude)

In [78]:
# SORT DATAFRAME DESCENDING
df.sort_values('Latitude', ascending=False)

,ID,Vehicle-ID,Line,Type,Latitude,Longitude,Timestamp,Date,Time
16528211,16528211,19775089,d,bus,7158.279,7158.279,2022-04-24 13:37:38.445930,2022-04-24,13:37:38.445930
25017226,25017226,19741475,2,tram,7158.279,7158.279,2022-04-29 17:59:42.887675,2022-04-29,17:59:42.887675
6428254,6428254,19760948,104,bus,7158.279,7158.279,2022-04-18 06:04:31.770293,2022-04-18,06:04:31.770293
15171417,15171417,19802765,1,tram,7158.279,7158.279,2022-04-23 12:18:04.384013,2022-04-23,12:18:04.384013
20355628,20355628,19819176,c,bus,7158.279,7158.279,2022-04-27 04:24:33.489169,2022-04-27,04:24:33.489169
...,...,...,...,...,...,...,...,...,...
7617396,7617396,19766067,602,bus,0.000,0.000,2022-04-19 06:49:36.167243,2022-04-19,06:49:36.167243
18566877,18566877,19741105,33,tram,0.000,0.000,2022-04-25 17:24:42.447676,2022-04-25,17:24:42.447676
17373534,17373534,19741092,33,tram,0.000,0.000,2022-04-25 05:16:12.900279,2022-04-25,05:16:12.900279
7415555,7415555,19766066,602,bus,0.000,0.000,2022-04-19 04:48:07.079381,2022-04-19,04:48:07.079381


In [79]:
# DROP OUTLIERS IN LATITUDE
df.drop(index=df[df['Latitude'] >= 60].index, inplace=True)
df.drop(index=df[df['Latitude'] <= 10].index, inplace=True)
# DROP OUTLIERS IN LONGITUDE
df.drop(index=df[df['Longitude'] >= 60].index, inplace=True)
df.drop(index=df[df['Longitude'] <= 10].index, inplace=True)

#### Print prepared dataframe

In [80]:
# PRINT DATAFRAME
df.sort_values('Latitude', ascending=False)

,ID,Vehicle-ID,Line,Type,Latitude,Longitude,Timestamp,Date,Time
3349756,3349756,19709911,20,tram,54.522713,21.743164,2022-04-15 13:33:51.133777,2022-04-15,13:33:51.133777
3348752,3348752,19709911,20,tram,54.522713,21.743164,2022-04-15 13:33:21.100799,2022-04-15,13:33:21.100799
3349254,3349254,19709911,20,tram,54.522713,21.743164,2022-04-15 13:33:36.105972,2022-04-15,13:33:36.105972
3348247,3348247,19709911,20,tram,54.499115,21.735200,2022-04-15 13:33:06.132351,2022-04-15,13:33:06.132351
3347237,3347237,19709911,20,tram,54.458454,21.720646,2022-04-15 13:32:36.109208,2022-04-15,13:32:36.109208
...,...,...,...,...,...,...,...,...,...
13005451,13005451,19766892,607,bus,51.012120,16.895754,2022-04-22 06:09:19.851066,2022-04-22,06:09:19.851066
13004464,13004464,19766892,607,bus,51.012040,16.889011,2022-04-22 06:08:34.588359,2022-04-22,06:08:34.588359
13003970,13003970,19766892,607,bus,51.012020,16.886427,2022-04-22 06:08:18.232878,2022-04-22,06:08:18.232878
13003476,13003476,19766892,607,bus,51.012010,16.885115,2022-04-22 06:08:02.921964,2022-04-22,06:08:02.921964


### (4) Visualise data

In [81]:
df['Line'].value_counts()

31     922155
710    641603
70     595992
17     523873
7      521849
        ...  
247     27220
147     22020
240     17120
250     16398
242     12448
Name: Line, Length: 93, dtype: int64

In [82]:
df['Type'].value_counts()

bus     16114766
tram     9486110
Name: Type, dtype: int64

#### Create new dataframe at specific date

In [83]:
# CREATE NEW DATAFRAME
time_df = pd.DataFrame()

# WRITE DATA AT SPECIFIC DATE IN NEW DATAFRAME
time_df = df[df['Date'] == '2022-04-20'] # HERE YOU CAN SET DATE

In [84]:
# COUNT ALL ROWS IN DATAFRAME
len(time_df.index)

1706166

In [85]:
# COUNT ENTRIES IN 'TYPE' COLUMN
time_df['Type'].value_counts()

bus     993495
tram    712671
Name: Type, dtype: int64

#### Create new dataframe for 'Type' and counts entries

In [86]:
# CREATE NEW DATAFRAME
type_df = pd.DataFrame()

# COUNTS ENTRIES OF 'Type' AT SPECIFIC TIME
type_df = time_df.groupby(['Type', 'Timestamp']).size().to_frame('Count')

# REMOVES SUB COLUMNS
type_df = type_df.reset_index()

# SHOWS AXES OF AN DATAFRAME
type_df.axes

[RangeIndex(start=0, stop=11211, step=1),
 Index(['Type', 'Timestamp', 'Count'], dtype='object')]

In [87]:
type_df

,Type,Timestamp,Count
0,bus,2022-04-20 00:00:07.981450,12
1,bus,2022-04-20 00:00:23.042499,16
2,bus,2022-04-20 00:00:37.978881,16
3,bus,2022-04-20 00:00:53.029423,15
4,bus,2022-04-20 00:01:07.978610,17
...,...,...,...
11206,tram,2022-04-20 22:49:26.644309,1
11207,tram,2022-04-20 22:49:41.639778,1
11208,tram,2022-04-20 22:49:56.728984,1
11209,tram,2022-04-20 22:50:11.810796,1


In [88]:
import plotly.express as px

fig = px.line(type_df, x="Timestamp", y="Count", color='Type')
fig.show()

#### Create new dataframe for a specific 'Line'

In [89]:
# CREATE NEW DATAFRAME
line_df = pd.DataFrame()

# WRITE DATA OF A SPECIFIC LINE IN NEW DATAFRAME
line_df = time_df[time_df['Line'] == '2'] # HERE YOU CAN SET LINE
line_df

,ID,Vehicle-ID,Line,Type,Latitude,Longitude,Timestamp,Date,Time
8982160,8982160,19741596,2,tram,51.124794,17.038363,2022-04-20 01:50:23.315102,2022-04-20,01:50:23.315102
8982195,8982195,19741596,2,tram,51.124794,17.038363,2022-04-20 01:50:38.284345,2022-04-20,01:50:38.284345
8982231,8982231,19741596,2,tram,51.124800,17.038366,2022-04-20 01:50:53.276852,2022-04-20,01:50:53.276852
8982267,8982267,19741596,2,tram,51.124800,17.038372,2022-04-20 01:51:08.279377,2022-04-20,01:51:08.279377
8982303,8982303,19741596,2,tram,51.124800,17.038372,2022-04-20 01:51:23.287097,2022-04-20,01:51:23.287097
...,...,...,...,...,...,...,...,...,...
10671635,10671635,19741887,2,tram,51.088370,17.028503,2022-04-20 22:35:26.768153,2022-04-20,22:35:26.768153
10671685,10671685,19741887,2,tram,51.088375,17.028526,2022-04-20 22:35:41.779726,2022-04-20,22:35:41.779726
10671746,10671746,19741887,2,tram,51.088406,17.028560,2022-04-20 22:35:56.675550,2022-04-20,22:35:56.675550
10671792,10671792,19741887,2,tram,51.088406,17.028560,2022-04-20 22:36:11.604891,2022-04-20,22:36:11.604891


In [90]:
# SHOW VALUES
line_df['Vehicle-ID'].value_counts()

19741549    264
19741778    263
19741886    261
19739749    255
19741506    255
           ... 
19741476     67
19741956     56
19741583     30
19747545      3
19747394      2
Name: Vehicle-ID, Length: 179, dtype: int64

#### Create new dataframe for a specific ‘Vehicle’

In [91]:
# CREATE NEW DATAFRAME
vehicle_df = pd.DataFrame()

# WRITE DATA OF A SPECIFIC LINE IN NEW DATAFRAME
vehicle_df = line_df[line_df['Vehicle-ID'] == '19741549'] # HERE YOU CAN SET VEHICLE-ID
vehicle_df

,ID,Vehicle-ID,Line,Type,Latitude,Longitude,Timestamp,Date,Time
9707884,9707884,19741549,2,tram,51.073753,17.006360,2022-04-20 11:22:39.830650,2022-04-20,11:22:39.830650
9708220,9708220,19741549,2,tram,51.073753,17.006360,2022-04-20 11:22:54.830524,2022-04-20,11:22:54.830524
9708548,9708548,19741549,2,tram,51.073753,17.006360,2022-04-20 11:23:09.833099,2022-04-20,11:23:09.833099
9708883,9708883,19741549,2,tram,51.073500,17.006004,2022-04-20 11:23:24.831375,2022-04-20,11:23:24.831375
9709221,9709221,19741549,2,tram,51.073494,17.005995,2022-04-20 11:23:40.074217,2022-04-20,11:23:40.074217
...,...,...,...,...,...,...,...,...,...
9803221,9803221,19741549,2,tram,51.102226,17.101788,2022-04-20 12:27:24.981249,2022-04-20,12:27:24.981249
9803614,9803614,19741549,2,tram,51.102130,17.102312,2022-04-20 12:27:39.965965,2022-04-20,12:27:39.965965
9804000,9804000,19741549,2,tram,51.101944,17.103416,2022-04-20 12:27:54.975726,2022-04-20,12:27:54.975726
9804385,9804385,19741549,2,tram,51.101692,17.105045,2022-04-20 12:28:10.217541,2022-04-20,12:28:10.217541


#### Create map for a specific 'Vehicle' at a specific 'Date'

In [92]:
# IMPORTS
import plotly.express as px

# SET MAPBOX TOKEN
px.set_mapbox_access_token(open(".mapbox_token").read())

# DESCRIBE MAP
fig = px.line_mapbox(data_frame=vehicle_df, lat="Latitude", lon="Longitude")

# LAYOUT ADJUSTMENTS
fig.update_layout(mapbox_zoom=11, mapbox_center_lat = 51.107883, mapbox_center_lon = 17.038538, margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

#### Create map with hexbins of counted entries at specific date

In [93]:
# IMPORTS
import plotly.figure_factory as ff
import plotly.express as px

# SET MAPBOX TOKEN
px.set_mapbox_access_token(open(".mapbox_token").read())

# DESCRIBE MAP
fig = ff.create_hexbin_mapbox(
    data_frame=time_df, lat="Latitude", lon="Longitude",
    nx_hexagon=50, opacity=0.5, labels={"color": "Point Count"},
    min_count=1,
)

# LAYOUT ADJUSTMENTS
fig.update_layout(mapbox_zoom=10, mapbox_center_lat = 51.107883, mapbox_center_lon = 17.038538, margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

#### Create plot for speed

In [94]:
# IMPORTS
from haversine import haversine, Unit
import pandas as pd

speed_df = pd.DataFrame()

first_index = vehicle_df['ID'].iloc[0]
store_lat = vehicle_df['Latitude'].iloc[1]
store_lon = vehicle_df['Longitude'].iloc[1]

# first = (vehicle_df.Latitude[5799852], vehicle_df.Longitude[5799852]) # (lat, lon)
# second = (vehicle_df.Latitude[5800048], vehicle_df.Longitude[5800048]) # (lat, lon)

#dist = (haversine(first, second))

# dist * 1000

vehicle_df = vehicle_df.astype({'ID':'int'})

#for index, row in vehicle_df.iterrows():
    #if index == first_index:
        #first = (vehicle_df.Latitude[index+1], vehicle_df.Longitude[index+1]) # (lat, lon)
        #second = (store_lat, store_lon) # (lat, lon)
        #speed = (((haversine(first, second)) * 1000) / 15) * 3.6
        #speed
    #if index > first_index:
        #first = (vehicle_df.Latitude[index], vehicle_df.Longitude[index]) # (lat, lon)
        #second = (vehicle_df.Latitude[index+1], vehicle_df.Longitude[index+1]) # (lat, lon)
        #speed = (((haversine(first, second)) * 1000) / 15) * 3.6
        #speed
